## Writing GRIB to file-pattern target

In [1]:
# get input GRIB data
import earthkit.data as ekd
ds = ekd.from_source("sample", "tuv_pl.grib")

tuv_pl.grib:   0%|          | 0.00/4.22k [00:00<?, ?B/s]

#### Using to_target() on the data object

In [2]:
# for each shortName-level combination a separate output file will be created
out_pattern = "_res_pattern_{shortName}_{level}.grib"

In [3]:
# writing only temperature fields
ds.sel(param="t").to_target("file-pattern", out_pattern)

# writing a whole fieldlist
ds.to_target("file-pattern", out_pattern)

# checking one output file
ekd.from_source("file", "_res_pattern_t_850.grib").ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,t,isobaricInhPa,850,20180801,1200,0,an,0,regular_ll


In [4]:
# setting GRIB keys for the output
ds.to_target("file-pattern", out_pattern, metadata={"date": 20250108}, bitsPerValue=8)

# checking one output file
ekd.from_source("file", "_res_pattern_t_850.grib").ls(extra_keys=["bitsPerValue"])

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType,bitsPerValue
0,ecmf,t,isobaricInhPa,850,20250108,1200,0,an,0,regular_ll,8


#### Using a Target object

In [5]:
# basic usage
target = ekd.create_target("file-pattern", out_pattern)
target.write(ds)
target.close()

# can be used as a context manager, no need to call close() in the end
with ekd.create_target("file-pattern", out_pattern) as target:
    target.write(ds)

# a fieldlist can be written field by field into the target
with ekd.create_target("file-pattern", out_pattern) as target:
    for f in ds:
        target.write(f)

#### Using encoders

These calls are equivalent.

In [6]:
# setting metadata and other GRIB keys for the output
ds.to_target("file-pattern", out_pattern, metadata={"date": 20250108}, bitsPerValue=8)

# explicitly specifying the encoder
ds.to_target("file-pattern", out_pattern, encoder="grib", metadata={"date": 20250108}, bitsPerValue=8)

# using an encoder object
encoder = ekd.create_encoder("grib", metadata={"date": 20250108})
ds.to_target("file-pattern", out_pattern, encoder=encoder,  bitsPerValue=8)

The same can be done with a target object.

In [7]:
encoder = ekd.create_encoder("grib", metadata={"date": 20250108})
with ekd.create_target("file-pattern", out_pattern) as target:
    target.write(ds, encoder=encoder,  bitsPerValue=8)